In [19]:
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries
from pathlib import Path
import json

In [3]:
API_KEY="JNGNBK43M85NTJ2Q"

In [4]:
class ScriptData:
        
    def __init__(self):
        self.script_list=[]
    
    #fetch the data
    def fetch_intraday_data(self,script):
        
        ts = TimeSeries(key='API_KEY',
                        indexing_type='date' )
        self.data, meta_data = ts.get_intraday(
                          symbol=script,interval='5min',
                          outputsize='full' )
    
    #convert the data to dataframe
    def convert_intraday_data(self,script): 
        
            df=pd.DataFrame.from_dict(self.data,orient='index')
            df.reset_index(inplace=True)
            df.rename(columns = {
                     'index':'timestamp','1. open':'open',
                     '2. high':'high','3. low':'low',
                     '4. close':'close','5. volume':'volume'}
                      , inplace = True)
            convert_dtype={
                            "open":float,
                            "high":float,
                            "low":float,
                            "close":float,
                            "volume":int
            }
            df['timestamp']=pd.to_datetime(
                            df['timestamp'], 
                            format='%Y-%m-%d %H:%M:%S')
            df.reset_index(drop=True, inplace=True)
            df=df.astype(convert_dtype)
            df.to_csv(script+'.csv',index=False) 
            self.script_list.append(script)
    
    def __getitem__(self,script):
        self.df=pd.read_csv(script+'.csv')
        return self.df
    
    def __contains__(self, script):
        return script in self.script_list

In [5]:
script_data=ScriptData()

In [6]:
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-02-13 20:00:00,94.35,94.40,94.28,94.40,13261
1,2023-02-13 19:55:00,94.35,94.40,94.35,94.35,6407
2,2023-02-13 19:50:00,94.28,94.39,94.28,94.35,2081
3,2023-02-13 19:45:00,94.42,94.44,94.29,94.29,2133
4,2023-02-13 19:40:00,94.47,94.47,94.35,94.35,917
...,...,...,...,...,...,...
3662,2023-01-17 04:25:00,91.82,91.82,91.60,91.72,3784
3663,2023-01-17 04:20:00,91.86,91.88,91.86,91.88,1980
3664,2023-01-17 04:15:00,91.90,91.97,91.87,91.87,3610
3665,2023-01-17 04:10:00,91.82,91.93,91.80,91.90,2016


In [7]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-02-13 20:00:00,153.6600,153.7000,153.6000,153.7000,7841
1,2023-02-13 19:55:00,153.6400,153.6700,153.6400,153.6600,4691
2,2023-02-13 19:50:00,153.6500,153.6500,153.6100,153.6300,3566
3,2023-02-13 19:45:00,153.6500,153.6700,153.6500,153.6700,2083
4,2023-02-13 19:40:00,153.6500,153.6600,153.6500,153.6600,6946
...,...,...,...,...,...,...
3778,2023-01-17 04:25:00,134.1557,134.1856,133.9460,133.9959,4045
3779,2023-01-17 04:20:00,134.2256,134.2555,134.1756,134.2555,2752
3780,2023-01-17 04:15:00,134.3654,134.3654,134.2755,134.2755,4748
3781,2023-01-17 04:10:00,134.3154,134.3454,134.2955,134.3454,3404


In [8]:
'NVDA' in script_data

False

In [9]:
'GOOGL' in script_data

True

In [10]:
'AAPL' in script_data

True

In [11]:
#generating indicator data
def indicator1(df:pd.DataFrame,timeperiod:int):
    
    indi_df = df['timestamp'].to_frame()
    
    #using rolling form pandas to calculate moving average
    indi_df['indicator']=df.iloc[:,4].rolling(
                                    timeperiod ).mean()
    return indi_df

In [12]:
indicator1(script_data['GOOGL'],5)

,timestamp,indicator
0,2023-02-13 20:00:00,NaN
1,2023-02-13 19:55:00,NaN
2,2023-02-13 19:50:00,NaN
3,2023-02-13 19:45:00,NaN
4,2023-02-13 19:40:00,94.348
...,...,...
3662,2023-01-17 04:25:00,91.700
3663,2023-01-17 04:20:00,91.730
3664,2023-01-17 04:15:00,91.772
3665,2023-01-17 04:10:00,91.820


In [13]:
indicator1(script_data['AAPL'],5)

,timestamp,indicator
0,2023-02-13 20:00:00,NaN
1,2023-02-13 19:55:00,NaN
2,2023-02-13 19:50:00,NaN
3,2023-02-13 19:45:00,NaN
4,2023-02-13 19:40:00,153.66400
...,...,...
3778,2023-01-17 04:25:00,134.11174
3779,2023-01-17 04:20:00,134.11374
3780,2023-01-17 04:15:00,134.13770
3781,2023-01-17 04:10:00,134.17964


In [14]:
class Strategy:
    
    def __init__(self,script):
        self.script=script       
        
    #getting script data from ScripData class    
    def get_script_data(self):
        script_data=ScriptData()
        df = script_data[self.script]
        df.rename(columns = {
                          'close':'close_data'},
                           inplace = True)
        ind_df = df['close_data'].to_frame()
        ind_df = indicator1(df,5)
        ind_df.rename( columns = {
                    'indicator' : 'indicator_data'},
                     inplace=True)
        self.scr_data=ind_df
        self.scr_data['close_data']=df['close_data']
        
                                       
    #generating signals dataframe
    def get_signals(self):
        signal = []
        position = False 
        data=self.scr_data
        
        for i in range(len(data)):
            if data['indicator_data'][i] > data['close_data'][i]:
                if position == False :
                    signal.append('BUY')
                    position = True
                else:
                    signal.append(np.nan)
            elif data['indicator_data'][i] < data['close_data'][i]:
                if position == True:
                    signal.append('SELL')
                    position = False
                else:
                    signal.append(np.nan)
            else:
                signal.append(np.nan)
        
        signal_data = data['timestamp'].to_frame()
        signal_data['signal'] = pd.Series(signal)
        signal_data.dropna(inplace=True,axis=0)
        return signal_data
    

In [15]:
stra = Strategy('GOOGL')
stra.get_script_data()
stra.get_signals()

,timestamp,signal
6,2023-02-13 19:30:00,BUY
10,2023-02-13 19:10:00,SELL
12,2023-02-13 19:00:00,BUY
15,2023-02-13 18:45:00,SELL
21,2023-02-13 18:15:00,BUY
...,...,...
3657,2023-01-17 04:50:00,BUY
3658,2023-01-17 04:45:00,SELL
3659,2023-01-17 04:40:00,BUY
3661,2023-01-17 04:30:00,SELL
